# General overview

-----------------------------------------------------------------------------------------------------------------
The following Jupyter notebook was created in order to automate the process of generating and visualizing forecasts for the macroeconomic indicators using historical time series retrieved using World Bank's Indicators API. Forecast will be generated using Auto ARIMA library which uses ARIMA, SARIMA and SARIMAX models under the hood and selects the one which suits best.

# Import libraries and set the display options
------------------------------------------------------------------------------------------------------------------

In [41]:
# General libraries
import json
from datetime import datetime

# Data preprocessing libraries and functions
import numpy as np
import pandas as pd
from retrieval_funcs import create_world_bank_api_url_string, retrieve_url_content
from preprocessing_funcs import convert_bytes_to_unicode, extract_dates_and_values_from_json

# Visualization function
from plotting_funcs import plot_historical_and_forecasted_values

# Modelling library
from pmdarima.arima import auto_arima

# Set pandas data display options
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Script parameters
------------------------------------------------------------------------------------------------------------------

<p style="color: #e60000; font-size:17px;">Please provide the country and economic indicator codes for which you would like to generate the forecast as well as year up to which it should be done. Downloaded time series and forecasted values will be saved to an '.xlsx' file and stored in the 'output' folder.</p>

In [167]:
COUNTRY_CODE = 'ger'
INDICATOR_CODE = 'EN.ATM.CO2E.PC'
PREDICT_UP_TO_YEAR_INCLUSIVE = 2025

# Retrieve and preprocess data 
------------------------------------------------------------------------------------------------------------------

In [168]:
# Create query string used to retrieve the data using World Bank's Indicators API
query_string = create_world_bank_api_url_string(COUNTRY_CODE, INDICATOR_CODE)

# Retrieve data 
data = retrieve_url_content(query_string)

# Convert retrieved data represented as bytes object to JSON
json_string = convert_bytes_to_unicode(data)
json_content = json.loads(json_string)

# Extract indicator_name, dates and values from JSON
indicator_name, dates, values = extract_dates_and_values_from_json(json_content)

# Create DataFrame object using dates and values
time_series_df = pd.DataFrame.from_dict({"Date": dates,
                                         "Value": values})

# Set index to Date column and change its type to datetime 
time_series_df.set_index('Date', inplace=True)
time_series_df.index = pd.to_datetime(time_series_df.index)

# Sort DataFrame years ascendingly
time_series_df.sort_values(by='Date', inplace=True)

IndexError: list index out of range

In [169]:
data

b'[{"message":[{"id":"120","key":"Invalid value","value":"The provided parameter value is not valid"}]}]'

### Missing values check

Now it's time to check if our time series has any missing values, and if so, deal with them.

There are many approaches to missing data imputation, i.e. filling the missing values with mean or median (might do the job in time independent datasets), using rolling average, training other models on the remaining explanatory variables and using them to predict the values for the missing column (applicable to multivariate datasets and very time consuming) or using some kind of interpolation. In case of the univariate series retrieved from the World Bank, we will stick to some kind of interpolation as suggested in the below article:

<b>https://medium.com/@drnesr/filling-gaps-of-a-time-series-using-python-d4bfddd8c460</b>

Once we are done, we will apply the Auto ARIMA library to help us choose the best forecasting model to the given time series. This will make the script useful for other economic indicators as well.

In [ ]:
# Get index of the first and last row with non-missing values
first_non_missing_row = time_series_df.first_valid_index()
last_non_missing_row = time_series_df.last_valid_index()

# If series has any missing values, fill in the gaps using 'time' interpolation
if time_series_df.isnull().sum().sum() > 0:
    # Subselect all rows up until the last non-missing one, if missing rows are at the end
    # of the time series, we will try to predict their values using Auto ARIMA model
    # instead of using Interpolation
    time_series_df = time_series_df[:last_non_missing_row]
    time_series_df['InterpolatedValue'] = time_series_df['Value'].interpolate(method='time')
else:
    pass

In [ ]:
# Verify the interpolation results by displaying the data
time_series_df.head(55)

In [ ]:
# Remove the 'Value' column as we will no longer need it
time_series_df.drop('Value', axis=1, inplace=True)

In [ ]:
# Make sure there are no missing values at the beginning of the time series - interpolation were not able
# to impute values for them and Auto ARIMA will raise an error in case of missing data

time_series_df = time_series_df[first_non_missing_row:]
time_series_df.head(10)

# Forecasting with Auto ARIMA
-----------------------------------------------------------------------------------------------------------------

Using Auto ARIMA library will speed up the forecasting process because the data preparation and parameter tuning processes for ARIMA, SARIMA and SARIMAX models end up being really time consuming. Auto ARIMA makes forecast preparation much simpler, because it:

<ul>
    <ul>
        <li> Performs data stationarity checks
        <li> Determines the <b>d</b> value which stands for the number of times the differencing operation has to be applied to the time series to make it stationary
        <li> Handles hyperparameter selection for models, hence saves us from doing this manually by looking at the ACF and PACF plots
        <li> Choses most accurate forecasting model for the given time series
    </ul>
</ul>


### Train the model

Usually, in all Machine Learning related projects we need to divide the dataset into training and validation subsets to verify whether the model's 'accurate' predictions are not a result of memorizing the dataset on which it was trained. It's also a good practice to prepare a test(holdout) subset which is used to verify the accuracy of the final model(chosen out of many). This practice is oftentimes used for Econometric models like ARMA, ARIMA or SARIMA as well, however taking into consideration the size of this dataset and the fact that around 40% of values were missing and had to be imputed, we will proceed by training the model on the whole dataset.

In [ ]:
model = auto_arima(time_series_df['InterpolatedValue'],
                   start_p=1,
                   start_q=1,
                   max_p=6,
                   max_q=6,
                   max_d=4,
                   maxiter=100,
                   max_order=None,
                   # Stepwise algo is less likely to over-fit than Grid Search
                   stepwise=True,
                   trace=True,
                   error_action='ignore',
                   suppress_warnings=True)

In [ ]:
model.summary()

### Forecast future values

In [ ]:
# Calculate number of periods for which the forecast needs to be generated
num_periods = PREDICT_UP_TO_YEAR_INCLUSIVE - last_non_missing_row.year

# Generate forecasts
forecast = model.predict(n_periods=num_periods)

In [ ]:
# Get time series beginning year timestamp and convert forecast horizon end year to Timestamp object
time_series_beginning = time_series_df.index[0]
forecast_end = pd.Timestamp(datetime.strptime(str(PREDICT_UP_TO_YEAR_INCLUSIVE + 1), '%Y'))


# Create DataFrame object storing historical and forecasted values
combined_df = pd.DataFrame({'Date': pd.date_range(start = time_series_beginning,
                                                  end=forecast_end,
                                                  freq='Y'),
                            'Value': time_series_df['InterpolatedValue'].tolist() + list(forecast)})

# Add 'Type' column to DataFrame to distinguish historical and forecasted values
combined_df['Type'] = ['historical' if i < len(time_series_df['InterpolatedValue'])
                       else 'forecasted' for i in range(len(combined_df))]

In [ ]:
# Load the country code to country name map from JSON file
with open('country_code_to_name.json') as json_file:
    code_to_name = json.load(json_file)

In [ ]:
combined_df.to_excel(f'output/{INDICATOR_CODE}-{code_to_name[COUNTRY_CODE]}.xlsx')

### Plot historical and forecasted values

In [ ]:
plot_historical_and_forecasted_values(combined_df,
                                      x='Date',
                                      y='Value',
                                      color_by_col='Type',
                                      indicator_code=INDICATOR_CODE,
                                      indicator_name = indicator_name,
                                      country=code_to_name[COUNTRY_CODE])